In [ ]:
import numpy as np
from astropy import units as u
from astropy.coordinates import SkyCoord
import scipy as sp
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
c = SkyCoord(ra=10.625*u.degree, dec=41.2*u.degree, frame='icrs')

def x (ra,dec):
    x = (np.cos(ra)*np.sin(dec))
    return x


def y (ra,dec):
    y = (np.cos(dec)*np.sin(ra))
    return y

In [ ]:
def cart2pol(x, y):
    rho = np.sqrt(x**2 + y**2)
    phi = np.arctan2(y, x)
    return(rho, phi)

def pol2cart(rho, phi):
    x = rho * np.cos(phi)
    y = rho * np.sin(phi)
    return(x, y)

In [ ]:
a= """0.003 0.358 0.567 1.066 0.323 0.0 0.0 0.721 1.159 -0.157 1.021 0.008 1.152 0.636 2.42 0.792 1.458 -0.456 0.0 0.0 0.768 1.662"""

In [ ]:
a= a.split(" ")
numLenses = 2
numImages = 4
oldData = np.zeros((numLenses,3,numImages))
newData = np.zeros((numLenses,3,numImages))

i=0
j=0
k=0
count=0;
while (i< numLenses):
    uncert= a[count]
    count=count+1;
    xChange=a[count]
    count=count+1
    yChange=a[count]
    count=count+1
    while (j< numImages):
        oldData[i][0][j] = float(a[count]) - float(xChange)
        count=count+1;
        oldData[i][1][j] = float(a[count]) - float(yChange)
        count=count+1;
        oldData[i][2][j]= float(uncert);
        j=j+1;
    i=i+1;
    j=0

    
i=0
j=0
k=0

while (i<numLenses):
    while (j<numImages):
        newData[i][0][j] = x(oldData[i][0][j], oldData[i][1][j])
        newData[i][1][j] = y(oldData[i][0][j], oldData[i][1][j])
        newData[i][2][j] = y(oldData[i][2][j], oldData[i][2][j])
        newData[i][0][j],newData[i][1][j] = cart2pol(newData[i][0][j], newData[i][1][j])
        #newData[i][0][j],newData[i][1][j] = pol2cart(newData[i][0][j], newData[i][1][j])
        j=j+1;
    i=i+1;
    j=0

Using polar coordinates centered on the lens galaxy, the combined lens potential can be written as

$$\phi (r, \theta) = b r f(\theta) + \frac{r^2}{2} (\gamma_c \cos{2 \theta} + \gamma_s \sin{2 \theta} )$$

where

$$f(\theta) = \left[ 1 - \epsilon \cos{2 ( \theta - \theta_0)} \right]^{1/2}$$

The deflection vector $\nabla \phi$ has cartesian components 

\begin{gather}
\nabla_x \phi = \frac{b}{f(\theta)} [\cos \theta - \epsilon \cos{(\theta - 2 \theta_0)}] + \gamma_c r \cos \theta + \gamma_s r \sin \theta \\
\nabla_y \phi = \frac{b}{f(\theta)} [\sin \theta - \epsilon \sin{(\theta - 2 \theta_0)}] + \gamma_s r \cos \theta - \gamma_c r \sin \theta
\end{gather}

The gravitational lens equation has the form

$$\vec{u} = \vec{x} - \nabla \phi (\vec{x})$$

which is really a set of two equations.
\begin{gather}
u = x - \nabla_x \phi (\vec{x}) \\
v = y - \nabla_y \phi (\vec{x})
\end{gather}
We need a penalty function $\chi^2$ that will determine the parameters $b, \epsilon, \gamma_c, \gamma_s, \theta_0$ in the lens potential $\phi$ and the parameters $u,v$, the position of the source. Let $\vec{x}_i, \sigma_i, \, i=0,1,2,3$ be the positions and uncertainties of the four images. Based on Keeton (2010, Gen.Rel.Grav., 42, 2151) we will define our $\chi^2$ function to be in the source plane. This eliminates the need for solving the lens equation (which is computationally expensive), and is a fine apprixmation given how small our uncertainties are. Let $\vec{\mu}_i = (\mu_i, \nu_i)$ be the position of the source as calculated from the lens equation using the $i$th image position. Then we can define our penalty function to be
$$ \chi^2 = \sum_{i=0}^3 \frac{1}{\sigma_i^2} \left( \vec{u} - \vec{\mu}_i \right)^2$$

In [ ]:
def f(theta, eps, theta0): # defining f(θ)
    return (1 - eps * np.cos(2 * (theta - theta0)))**(1/2)


def dxphi(rtheta, b, eps, gc, gs, theta0): # deflection vector x component
    r, theta = rtheta
    return b / f(theta, eps, theta0) * (np.cos(theta) - eps * np.cos( theta - 2 * theta0)) \
        + gc * r * np.cos(theta) + gs * r * np.sin(theta)


def dyphi(rtheta, b , eps, gc, gs, theta0): #deflection vector y component
    r, theta = rtheta
    return b / f(theta, eps, theta0) * (np.sin(theta) - eps * np.sin( theta - 2 * theta0)) \
        + gs * r * np.cos(theta) - gc * r * np.sin(theta)


def calcsource(params, rtheta): # calculates the source given the ϕ-params and an image position
    b, eps, gc, gs, theta0, _, _ = params
    r, theta = rtheta
    x, y = pol2cart(r, theta)
    mu = x - dxphi(rtheta, b, eps, gc, gs, theta0)
    nu = y - dyphi(rtheta, b, eps, gc, gs, theta0)
    return (mu, nu)

    
def lnprob(params, rthetasigma): # our χ^2 function
    b, eps, gc, gs, theta0, u, v = params
    r, theta, sigma = rthetasigma
    chi2 = 0
    for i in range(3):
        mu, nu = calcsource(params, rtheta[i])
        chi2 += ((u - mu)**2 + (v - nu)**2) / sigma[i]**2
    return chi2

In [ ]:
ndim = 7
nwalk = 16
nburn = 200
nmain = 1000

# might be better to use this segment if we have a better idea of what the low and high values might be for each parameter
# generate random starting points
# p0 = np.zeros((nwalk,ndim))
# for iwalk in range(nwalk):
#     p0[iwalk,0] = np.random.uniform(low=,high=,)
#     p0[iwalk,1] = np.random.uniform(low=,high=)
#     p0[iwalk,2] = np.random.uniform(low=,high=)
#     p0[iwalk,3] = np.random.uniform(low=,high=)
#     p0[iwalk,4] = np.random.uniform(low=,high=)
#     p0[iwalk,5] = np.random.uniform(low=,high=)
#     p0[iwalk,6] = np.random.uniform(low=,high=)
    
# pstart = np.zeros(ndim)
p0 = np.array(np.random.rand(ndim) for i in range(nwalk)) 
# p0 = np.array([ pstart+1.0e-4*np.random.normal(size=ndim) for iwalk in range(nwalk) ])


sampler = emcee.EnsembleSampler(nwalk,ndim,lnprob) 

# burn-in run
pos,prob,state = sampler.run_mcmc(p0,nburn)

sampler.reset()

# main run
res = sampler.run_mcmc(pos,nmain)
samples = sampler.chain.reshape((-1,ndim))

In [ ]:
f = corner.corner(samples,show_titles=True,labels=('p0','p1','p2','p3','p4','p5','p6'))
f.show()